In [3]:
import os
import shutil
import pandas as pd
import numpy as np

# Charger le DataFrame avec les informations sur les photos

df_photos = pd.read_json('C:/Users/david/Dropbox/Openclassrooms/IA/Projet 6/Photos/photos.json', lines=True)
# Définir les répertoires
base_dir = r'C:/Users/david/Dropbox/Openclassrooms/IA/Projet 6/Photos/photos'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Créer les répertoires de base s'ils n'existent pas
os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

# Itérer sur chaque label unique pour créer les sous-dossiers (food, outside, inside, etc.)
for label in df_photos['label'].unique():
    os.makedirs(os.path.join(train_dir, label), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, label), exist_ok=True)

# Mélanger les données
df_photos_shuffled = df_photos.sample(frac=1, random_state=42).reset_index(drop=True)

# Fonction pour sélectionner 200 images par classe
def select_limited_images_per_class(df, limit=200):
    limited_df = df.groupby('label').apply(lambda x: x.sample(n=min(len(x), limit), random_state=42)).reset_index(drop=True)
    return limited_df

# Séparer en 80% Train et 20% Validation
train_split = int(0.8 * len(df_photos_shuffled))
df_train = df_photos_shuffled[:train_split]
df_validation = df_photos_shuffled[train_split:]

# Limiter à 200 images par classe
df_train_limited = select_limited_images_per_class(df_train, limit=200)
df_validation_limited = select_limited_images_per_class(df_validation, limit=200)

# Fonction pour déplacer les fichiers
def move_photos(df, destination_dir):
    for _, row in df.iterrows():
        photo_id = row['photo_id'] + '.jpg'  # Ajouter l'extension .jpg au photo_id
        label = row['label']
        
        # Chemin source
        source_path = os.path.join(base_dir, photo_id)
        
        # Chemin de destination basé sur le label (classe)
        dest_path = os.path.join(destination_dir, label, photo_id)
        
        # Déplacer la photo
        if os.path.exists(source_path):  # Vérifier que le fichier existe bien
            shutil.move(source_path, dest_path)
        else:
            print(f"File {source_path} not found.")

# Déplacer les photos vers le répertoire Train
move_photos(df_train_limited, train_dir)

# Déplacer les photos vers le répertoire Validation
move_photos(df_validation_limited, validation_dir)

print("200 photos par classe déplacées avec succès.")


200 photos par classe déplacées avec succès.
